<a href="https://colab.research.google.com/github/honicky/deep-log-analysis/blob/main/Pythia%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pythia Analysis - train small models on HDFS data

* use tokenized version of preprocessed HDFS events
* start with very small pythia models, test increasing size
* start with fine-tuning, then consider resetting weights and training from scratch
* experiment with different tokenizers
  * https://chatgpt.com/share/67448f53-29a0-800f-9913-af22d6ed0894

Interesting note: [Understanding LLM Embeddings for Regression](https://arxiv.org/pdf/2411.14708) discusses how using textual embeddings of primarily numerical data is actually surprisingly effective. This supports the hypothesis here that we can use a pretrained model to use embeddings for the log data rather than training a specialized model for logs which are dominated by numbers.

In [44]:
try:
  from google.colab import userdata

  !git clone https://github.com/honicky/deep-log-analysis.git
  !mv deep-log-analysis/* .
  !rm -rf deep-log-analysis
except:
  pass

In [45]:
try:
    import logparser.Drain as Drain
except ImportError:
    %pip install requests git+https://github.com/logpai/logparser

%pip install transformers torch torchvision torchaudio wandb python-dotenv datasets

Python(58102) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
%load_ext autoreload
%autoreload 2
import dataloaders as dl
%autoreload 2
import model_utils


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load secrets

If we are in colab, we get them from the `userdata` module, otherwise we get them from a .env file


In [47]:
import os
try:
  from google.colab import userdata
  os.environ["HF_WRITE_TOKEN"] = userdata.get('HF_WRITE_TOKEN')
  os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')
except ImportError:
  from dotenv import load_dotenv
  load_dotenv()


In [66]:
base_model_name = "EleutherAI/pythia-70m"  # @param ["EleutherAI/pythia-14m", "EleutherAI/pythia-70m"]
model_name = f"{base_model_name.split('/')[-1]}-hdfs-logs"


# Download and unzip the HDFS dataset

The functions check if the data is already downloaded and unzipped, and only download and unzip if they are not present.


In [67]:
from transformers import GPTNeoXTokenizerFast
tokenizer = GPTNeoXTokenizerFast.from_pretrained(base_model_name)
tokenizer.add_special_tokens({"additional_special_tokens": ["<|sep|>"]})
tokenizer.sep_token = "<|sep|>"
tokenizer.sep_token_id
tokenizer.pad_token_id = tokenizer.eos_token_id # no pad token in default tokenizer, so add it here for collating / training


Double check that the tokenizer properly encodes the new special token

In [68]:

tokenizer.encode("<|sep|>")


[50277]

Review then tokenizer configuration, again to ensure the new special token is included


In [69]:
tokenizer

GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-70m', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'sep_token': '<|sep|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|sep|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50256: AddedToken("                      ", rstrip=False, lstrip=False, 

In [70]:
import torch

from transformers import GPTNeoXForCausalLM

def get_model():

    model = GPTNeoXForCausalLM.from_pretrained(base_model_name)
    model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=64)
    model.get_input_embeddings().weight.data.shape

    return model

model = get_model()

In [71]:
from datasets import load_dataset
import pandas as pd

# Load the dataset from the hub
dataset_dict = load_dataset("honicky/hdfs-logs-encoded-blocks")

# Convert the train split to a pandas DataFrame
train_df = dataset_dict['train'].to_pandas()
val_df = dataset_dict['validation'].to_pandas()

In [72]:
import model_utils
globals().update(model_utils.training_params())



In [73]:
print(f"using BATCH_SIZE = {BATCH_SIZE}")
print(f"using MAX_LENGTH = {MAX_LENGTH}")
print(f"using LEARNING_RATE = {LEARNING_RATE}")
print(f"using NUM_EPOCHS = {NUM_EPOCHS}")

using BATCH_SIZE = 4
using MAX_LENGTH = 405
using LEARNING_RATE = 0.0001
using NUM_EPOCHS = 1


In [74]:
import os, wandb

wandb.login(key=os.getenv("WANDB_API_KEY"))


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [75]:

from model_utils import print_memory_stats, get_gpu_memory_metrics, clear_memory


In [76]:
# Create DataLoader
class HDFSDataset(torch.utils.data.Dataset):
    def __init__(self, encoded_blocks, max_length):
        self.tokenized_blocks = encoded_blocks
        self.max_length = max_length

    def __len__(self):
        return len(self.tokenized_blocks)

    def __getitem__(self, idx):
        tokens = self.tokenized_blocks.iloc[idx]['tokenized_block']
        # Truncate if needed
        if len(tokens) > self.max_length:
            tokens = tokens[:self.max_length]

        # Convert to tensor and pad
        input_ids = torch.tensor(tokens, dtype=torch.long)
        attention_mask = torch.ones_like(input_ids)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }

def create_dataloader(encoded_pdf, tokenizer):

    dataset = HDFSDataset(encoded_pdf, MAX_LENGTH)
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=lambda x: {
            'input_ids': torch.nn.utils.rnn.pad_sequence(
                [item['input_ids'] for item in x],
                batch_first=True,
                padding_value=tokenizer.pad_token_id if tokenizer.pad_token_id else 0
            ),
            'attention_mask': torch.nn.utils.rnn.pad_sequence(
                [item['attention_mask'] for item in x],
                batch_first=True,
                padding_value=0
            )
        }
    )

    return dataloader

dataloader = create_dataloader(train_df, tokenizer)

In [77]:
val_df.head()

,event_encoded,tokenized_block,block_id,label,__index_level_0__
0,<|sep|>0 /10.251.125.193:49078 /10.251.125.193...,"[50277, 17, 1227, 740, 15, 21451, 15, 9312, 15...",blk_8706546487798466885,Normal,370570
1,<|sep|>0 /10.251.74.192:36984 /10.251.74.192:5...,"[50277, 17, 1227, 740, 15, 21451, 15, 3566, 15...",blk_3164806166289090589,Normal,387094
2,<|sep|>0 /10.251.67.113:44473 /10.251.67.113:5...,"[50277, 17, 1227, 740, 15, 21451, 15, 2251, 15...",blk_6334862664379948501,Normal,524461
3,<|sep|>0 /10.250.15.67:36719 /10.250.15.67:500...,"[50277, 17, 1227, 740, 15, 9519, 15, 1010, 15,...",blk_-4209139676364491359,Normal,491282
4,<|sep|>0 /10.251.111.228:56317 /10.251.111.228...,"[50277, 17, 1227, 740, 15, 21451, 15, 10768, 1...",blk_-7362312881779468190,Normal,671


In [78]:

print_memory_stats()


 Memory Status:
├── Allocated: 0.26 GB (actively used by tensors)
├── Reserved:  1.02 GB (held by driver)
├── Cached:    0.75 GB (reserved - allocated)
└── System Available: 2.68 GB


In [79]:
clear_memory()

In [64]:
import numpy as np

def evaluate_model(model, dataloader, device):
    """
    Evaluate the model on the provided dataloader with detailed perplexity metrics
    """
    model.eval()
    total_loss = 0
    num_batches = 0
    all_perplexities = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids
            )

            # Calculate per-token perplexity
            loss = outputs.loss
            batch_perplexity = torch.exp(outputs.logits[..., :-1, :].log_softmax(-1).gather(
                -1, input_ids[..., 1:].unsqueeze(-1)
            ).squeeze(-1) * -1)

            # Mask out padding tokens
            mask = attention_mask[..., 1:].bool()
            valid_perplexities = batch_perplexity[mask].cpu().numpy()
            all_perplexities.extend(valid_perplexities.tolist())

            total_loss += loss.item()
            num_batches += 1

            wandb.log({
                "eval/batch_loss": loss.item(),
                **get_gpu_memory_metrics()
            })

    # Calculate percentiles
    percentiles = np.percentile(all_perplexities, [50, 75, 90, 95, 99, 100])

    # Log to terminal
    print("\nPerplexity Percentiles:")
    print(f"50th:       {percentiles[0]:.2f}")
    print(f"75th:       {percentiles[1]:.2f}")
    print(f"90th:       {percentiles[2]:.2f}")
    print(f"95th:       {percentiles[3]:.2f}")
    print(f"99th:       {percentiles[4]:.2f}")
    print(f"Max (100th): {percentiles[5]:.2f}")

    # Log to wandb
    wandb.log({
        "eval/avg_loss": total_loss / num_batches,
        "eval/perplexity_p50": percentiles[0],
        "eval/perplexity_p75": percentiles[1],
        "eval/perplexity_p90": percentiles[2],
        "eval/perplexity_p95": percentiles[3],
        "eval/perplexity_p99": percentiles[4],
        "eval/perplexity_max": percentiles[5],
    })

    return total_loss / num_batches

def train_model(model, dataloader, optimizer, device, steps=None, start_batch=0):
    """
    Train the model for a specified number of steps or until the dataloader is exhausted.

    Args:
        model: The model to train
        dataloader: DataLoader containing the training data
        optimizer: The optimizer to use
        device: The device to train on
        steps (int, optional): Number of steps to train. If None, train on all remaining batches
        start_batch (int): The batch index to start from (for resuming training)

    Returns:
        tuple: (global_step, batch_idx) - The current global step and batch index for resuming
    """
    model.train()
    global_step = start_batch
    total_loss = 0

    for batch_idx, batch in enumerate(dataloader, start=start_batch):
        # Check if we've reached the requested number of steps
        if steps is not None and (batch_idx - start_batch) >= steps:
            break

        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=input_ids
        )

        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print progress every 100 batches
        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

        wandb.log({
            "train/batch_loss": loss.item(),
            "train/batch": batch_idx,
            **get_gpu_memory_metrics()
        }, step=global_step)

        global_step += 1

    avg_loss = total_loss / (batch_idx - start_batch + 1)
    print(f"Training complete. Average loss: {avg_loss:.4f}")
    wandb.log({
        "train/avg_loss": avg_loss,
    })

    return global_step, batch_idx

In [65]:
# Move model to MPS device if available, otherwise CPU
model = get_model().to(device)

# Set up optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

wandb.init(
    project="log-analysis-pythia",
    config={
        "batch_size": BATCH_SIZE,
        "max_length": MAX_LENGTH,
        "learning_rate": LEARNING_RATE,
        "epochs": NUM_EPOCHS,
        "model": "pythia-14m",
    }
)


Python(59858) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Python(59877) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [80]:
print_memory_stats()


 Memory Status:
├── Allocated: 0.26 GB (actively used by tensors)
├── Reserved:  1.01 GB (held by driver)
├── Cached:    0.75 GB (reserved - allocated)
└── System Available: 2.69 GB


In [81]:
len(dataloader)

115012

In [82]:
try:
  current_batch = 0
  for i in range(int(len(dataloader)/2000)+1):
      current_step, current_batch = train_model(model, dataloader, optimizer, device, steps=2000, start_batch=current_batch)

      eval_dataloader = create_dataloader(val_df[:10*BATCH_SIZE], tokenizer)
      evaluate_model(model, eval_dataloader, device)
except:
  # print stack trace
  import traceback
  traceback.print_exc()


Traceback (most recent call last):
  File "/var/folders/1w/njpw08_93h73169nbj9b9z700000gp/T/ipykernel_9929/1227516864.py", line 4, in <module>
    current_step, current_batch = train_model(model, dataloader, optimizer, device, steps=2000, start_batch=current_batch)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/1w/njpw08_93h73169nbj9b9z700000gp/T/ipykernel_9929/4241282003.py", line 96, in train_model
    outputs = model(
              ^^^^^^
  File "/Users/rj/personal/log_analysis/venv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rj/personal/log_analysis/venv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rj/perso

In [88]:

# Save model to HuggingFace Hub
model.push_to_hub(
    f"honicky/{model_name}",
    token=os.environ["HF_WRITE_TOKEN"],
    commit_message=f"Trained {current_step} steps"
)

# Save tokenizer with the added special tokens
tokenizer.push_to_hub(
    f"honicky/{model_name}",
    token=os.environ["HF_WRITE_TOKEN"],
    commit_message="Tokenizer with added special tokens for HDFS logs"
)

# Save model config and training details
with open("README_model.md", "w") as f:
    f.write(f"""---
language: en
tags:
- log-analysis
- pythia
- hdfs
license: mit
datasets:
- honicky/log-analysis-hdfs-preprocessed
metrics:
- cross-entropy
- perplexity
base_model: {base_model_name}
---

# {model_name}

Fine-tuned Pythia-14m model for HDFS log analysis, specifically for anomaly detection.

## Model Description

This model is fine-tuned from `{base_model_name}` for analyzing HDFS log sequences. It's designed to understand and predict patterns in
HDFS log data so that we can detect anomalies using the perplexity of the log sequence. THhe HDFS sequence is handy because it has labels
so we can use it to validate that the model can predict anomalies.

We will use this model to understand the ability of a small model to predict anomalies in a specific dataset.  We will study model scale
and experiment with tokenization, intialization, data set size, etc. to find a configuration that is minimal in size and fast, but can
effectively predict anomalies.  We will then attempt build a model that is more robust to different log formats.

- Huggingface Model: [honicky/pythia-14m-hdfs-logs](https://huggingface.co/honicky/pythia-14m-hdfs-logs)

## Training Details
- Base model: {base_model_name}
- Dataset: https://zenodo.org/records/8196385/files/HDFS_v1.zip?download=1 + preprocessed data at honicky/log-analysis-hdfs-preprocessed
- Batch size: {BATCH_SIZE}
- Max sequence length: {MAX_LENGTH}
- Learning rate: {LEARNING_RATE}
- Training steps: {current_step}
- Weights and Biases run: {wandb.run.url}


## Special Tokens
- Added `<|sep|>` token for event ID separation

## Intended Use
This model is intended for:
- Analyzing HDFS log sequences
- Detecting anomalies in log patterns
- Understanding system behavior through log analysis

## Limitations
- Model is specifically trained on HDFS logs and may not generalize to other log formats
- Limited to the context window size of {MAX_LENGTH} tokens


""")


# Push README
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="README_model.md",
    path_in_repo="README_model.md",
    repo_id=f"honicky/{model_name}",
    token=os.environ["HF_WRITE_TOKEN"],
    commit_message="Add model documentation"
)

README.md:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/56.3M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/honicky/pythia-14m-hdfs-logs/commit/ebff8b9ef46e6de457d4b0af78200da41a5208d3', commit_message='Add model documentation', commit_description='', oid='ebff8b9ef46e6de457d4b0af78200da41a5208d3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/honicky/pythia-14m-hdfs-logs', endpoint='https://huggingface.co', repo_type='model', repo_id='honicky/pythia-14m-hdfs-logs'), pr_revision=None, pr_num=None)